In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Loading data 
b_data = pd.read_json('business.json', lines=True)
b_sub = b_data.sample(frac=0.01)

In [3]:
#Filtering Data

b_sub = b_sub[b_sub.is_open == 1]

In [4]:
labels = ['Restaurants', 'restaurants','Restaurant', 'restaurant','bar','Bar','bars','Bars',
          'Food','food','Foods','foods','Dinner','dinner','Breakfast','breakfast']
def is_restaurant(x): 
    for label in labels: 
        if label in x:
            return True
    return False 

In [20]:
# Extracting restaurants
b_filt = b_sub[b_sub.categories.apply(is_restaurant)]

In [21]:
#Explanation: in the original dataset, the actual characteristics of the restaurants are included in a column titled
#'attributes'. Within this column are different categorical variables that describe the atmosphere 

attr_dict={}

for item in b_filt.attributes.iteritems():
    newatts = item[1]
    for key,value in newatts.items():
        if key not in attr_dict.keys():
            attr_dict[key] = [value]
        else:
            if value not in attr_dict[key]:
                if type(attr_dict[key][0]) is not dict:#this is to stop the list from getting all possible
                    #dictionary entries for the attribute dict
                    attr_dict[key].append(value)

print("The Attriubte Column Expanded:")
print(attr_dict)

The Attriubte Column Expanded:
{'RestaurantsTableService': [False, True], 'GoodForMeal': [{'dessert': False, 'latenight': False, 'lunch': True, 'dinner': False, 'breakfast': False, 'brunch': False}], 'Alcohol': ['none', 'beer_and_wine', 'full_bar'], 'Caters': [True, False], 'HasTV': [False, True], 'RestaurantsGoodForGroups': [True, False], 'NoiseLevel': ['average', 'quiet', 'loud', 'very_loud'], 'WiFi': ['no', 'free'], 'RestaurantsAttire': ['casual', 'dressy', 'formal'], 'RestaurantsReservations': [False, True], 'OutdoorSeating': [False, True], 'BusinessAcceptsCreditCards': [True, False], 'RestaurantsPriceRange2': [1, 2, 3, 4], 'BikeParking': [True, False], 'RestaurantsDelivery': [False, True], 'Ambience': [{'romantic': False, 'intimate': False, 'classy': False, 'hipster': False, 'divey': False, 'touristy': False, 'trendy': False, 'upscale': False, 'casual': True}], 'RestaurantsTakeOut': [True, False], 'GoodForKids': [True, False], 'DriveThru': [True, False], 'BusinessParking': [{'gara

In [22]:
subattr_list =[]

for key,value in attr_dict.items():
    if type(value[0])==dict:
        #This triggers the subdict protocol
        prefix = key
        subdict = value[0]
        for key2, value2 in subdict.items():
            label = prefix +"_"+key2
            subattr_list.append(label)
    elif type(value[0])==str:
        #This triggers a non-binary attribute, but not a subdict
        for item in value:
            prefix = key
            label = prefix+'_'+str(item)
            subattr_list.append(label)
    else:
        #this means thatt the condition must be binary
        subattr_list.append(key)

columns=[]
for x in subattr_list:
    if 'Hair' not in x:
        columns.append(x)
print(columns)

['RestaurantsTableService', 'GoodForMeal_dessert', 'GoodForMeal_latenight', 'GoodForMeal_lunch', 'GoodForMeal_dinner', 'GoodForMeal_breakfast', 'GoodForMeal_brunch', 'Alcohol_none', 'Alcohol_beer_and_wine', 'Alcohol_full_bar', 'Caters', 'HasTV', 'RestaurantsGoodForGroups', 'NoiseLevel_average', 'NoiseLevel_quiet', 'NoiseLevel_loud', 'NoiseLevel_very_loud', 'WiFi_no', 'WiFi_free', 'RestaurantsAttire_casual', 'RestaurantsAttire_dressy', 'RestaurantsAttire_formal', 'RestaurantsReservations', 'OutdoorSeating', 'BusinessAcceptsCreditCards', 'RestaurantsPriceRange2', 'BikeParking', 'RestaurantsDelivery', 'Ambience_romantic', 'Ambience_intimate', 'Ambience_classy', 'Ambience_hipster', 'Ambience_divey', 'Ambience_touristy', 'Ambience_trendy', 'Ambience_upscale', 'Ambience_casual', 'RestaurantsTakeOut', 'GoodForKids', 'DriveThru', 'BusinessParking_garage', 'BusinessParking_street', 'BusinessParking_validated', 'BusinessParking_lot', 'BusinessParking_valet', 'DogsAllowed', 'ByAppointmentOnly', '

In [32]:
def get_attr(x, col):
    
    newattribute = 'GoodForMeal_dessert'
    
    if '_' in newattribute:
        temp=newattribute.split('_')
        key1=temp[0]
        key2=temp[1]
        
        if key1 in x.keys():
            if key2 in x[key1].keys():
                return x[key1][key2]
        else:
            return False
    else:
        key1 = newattribute
        if key1 in x.keys():
            return x[key1]
        else:
            return False
    
    

In [34]:
for col in columns:
    b_filt[col] = b_filt.attributes.apply(lambda x: get_attr(x,col))
b_filt.head()

//anaconda/envs/py3k/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,...,AgesAllowed_19plus,AgesAllowed_18plus,AcceptsInsurance,DietaryRestrictions_dairy-free,DietaryRestrictions_gluten-free,DietaryRestrictions_vegan,DietaryRestrictions_kosher,DietaryRestrictions_halal,DietaryRestrictions_soy-free,DietaryRestrictions_vegetarian
118400,10000 W Happy Valley Rd,"{'RestaurantsTableService': False, 'GoodForMea...",UHlHCV8chJYLHA-C4YeLnA,"[Fast Food, Restaurants, Mexican]",Peoria,"{'Monday': '10:00-22:00', 'Tuesday': '10:00-22...",1,33.712017,-112.274735,El Pollo Loco,...,False,False,False,False,False,False,False,False,False,False
151826,Plieninger Str. 3/1,"{'BusinessAcceptsCreditCards': False, 'Restaur...",fP_KKgMBCw6UDhWZsbRC2Q,"[Restaurants, Swabian, International, German]",Ostfildern,{},1,48.707210,9.261434,Lamm,...,False,False,False,False,False,False,False,False,False,False
29350,606 S Whitney Way,"{'BusinessParking': {'garage': False, 'street'...",8WTwjhS_2lR2yiUFU83Tcg,"[Food, Convenience Stores, Drugstores, Cosmeti...",Madison,"{'Monday': '0:00-0:00', 'Tuesday': '0:00-0:00'...",1,43.053162,-89.474455,Walgreens,...,False,False,False,False,False,False,False,False,False,False
49410,444 Yonge St,"{'RestaurantsTableService': False, 'GoodForMea...",CanfdSFEs_o_h9SBvxyXlg,"[Chinese, Restaurants]",Toronto,{},1,43.661006,-79.383213,Duck,...,False,False,False,False,False,False,False,False,False,False
25719,9240 W Northern Ave,"{'Alcohol': 'beer_and_wine', 'HasTV': True, 'N...",6rl2DTYflL0t1_oL4KudYw,"[Pizza, Restaurants, Sandwiches, Italian]",Peoria,"{'Monday': '10:30-21:00', 'Tuesday': '10:30-21...",1,33.552659,-112.258322,Streets of New York,...,False,False,False,False,False,False,False,False,False,False
